<a href="https://colab.research.google.com/github/somura496/2025research/blob/main/Vectorize-by-SentenceTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SentenceTransformerによる文章ベクトル化

##引用コードサンプル

In [1]:
from sentence_transformers import SentenceTransformer, util

# モデルのロード
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

# 比較する文章
sentence1 = "自然言語処理は非常に興味深い分野です。"
sentence2 = "自然言語処理には多くの挑戦がありますが、面白いです。"

# 文章をベクトルに変換
embeddings1 = model.encode(sentence1, convert_to_tensor=True)
embeddings2 = model.encode(sentence2, convert_to_tensor=True)

# コサイン類似度の計算
cosine_score = util.pytorch_cos_sim(embeddings1, embeddings2)[0][0]

print(f"文章1と文章2の類似度: {cosine_score}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


文章1と文章2の類似度: 0.9014313220977783


##実行するやつ

### 環境構築

In [2]:
# huggingface transformer のインストール
# - transformers : 主たるモジュール（モデルやトークナイザのダウンロード)
# - datasets : HuggingFaceで、データセットを扱うためのモジュール
# https://huggingface.co/docs/transformers/installation
! pip install transformers datasets

# 東北大学の日本語用BERT使用に必要なパッケージをインストール
! pip install fugashi ipadic

In [3]:
import numpy as np
import pandas as pd

In [4]:
# Hugging Face (Transformers) 関連のモジュール -> なんか上手くいかない
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# from transformers import TrainingArguments, Trainer
# from datasets import Dataset, load_metric

ImportError: cannot import name 'load_metric' from 'datasets' (/usr/local/lib/python3.12/dist-packages/datasets/__init__.py)

### dfwrimeの取得と書き出し

In [6]:
! wget https://github.com/ids-cv/wrime/raw/master/wrime-ver1.tsv
df_wrime = pd.read_table('wrime-ver1.tsv')
df_wrime.head(2)

--2025-11-10 07:22:36--  https://github.com/ids-cv/wrime/raw/master/wrime-ver1.tsv
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ids-cv/wrime/master/wrime-ver1.tsv [following]
--2025-11-10 07:22:38--  https://raw.githubusercontent.com/ids-cv/wrime/master/wrime-ver1.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9487235 (9.0M) [text/plain]
Saving to: ‘wrime-ver1.tsv.2’

wrime-ver1.tsv.2    100%[===================>]   9.05M  --.-KB/s    in 0.1s    

2025-11-10 07:22:38 (78.6 MB/s) - ‘wrime-ver1.tsv.2’ saved [9487235/9487235]



,Sentence,UserID,Datetime,Train/Dev/Test,Writer_Joy,Writer_Sadness,Writer_Anticipation,Writer_Surprise,Writer_Anger,Writer_Fear,...,Reader3_Disgust,Reader3_Trust,Avg. Readers_Joy,Avg. Readers_Sadness,Avg. Readers_Anticipation,Avg. Readers_Surprise,Avg. Readers_Anger,Avg. Readers_Fear,Avg. Readers_Disgust,Avg. Readers_Trust
0,ぼけっとしてたらこんな時間｡チャリあるから食べにでたいのに…,1,2012/07/31 23:48,train,0,1,2,1,1,0,...,1,0,0,2,0,0,0,0,0,0
1,今日の月も白くて明るい。昨日より雲が少なくてキレイな? と立ち止まる帰り道｡チャリなし生活も...,1,2012/08/02 23:09,train,3,0,3,0,0,0,...,0,1,1,0,0,2,0,0,0,0


### 8感情タグの付与

In [7]:
# Plutchikの8つの基本感情
emotion_names = ['Joy', 'Sadness', 'Anticipation', 'Surprise', 'Anger', 'Fear', 'Disgust', 'Trust']
emotion_names_jp = ['喜び', '悲しみ', '期待', '驚き', '怒り', '恐れ', '嫌悪', '信頼']  # 日本語版
num_labels = len(emotion_names)

# readers_emotion_intensities 列を生成する
# \"Avg. Readers_*\" の値をlist化したもの
df_wrime['readers_emotion_intensities'] = df_wrime.apply(lambda x: np.array([x['Avg. Readers_' + name] for name in emotion_names]), axis=1)
df_wrime.head(2)

,Sentence,UserID,Datetime,Train/Dev/Test,Writer_Joy,Writer_Sadness,Writer_Anticipation,Writer_Surprise,Writer_Anger,Writer_Fear,...,Reader3_Trust,Avg. Readers_Joy,Avg. Readers_Sadness,Avg. Readers_Anticipation,Avg. Readers_Surprise,Avg. Readers_Anger,Avg. Readers_Fear,Avg. Readers_Disgust,Avg. Readers_Trust,readers_emotion_intensities
0,ぼけっとしてたらこんな時間｡チャリあるから食べにでたいのに…,1,2012/07/31 23:48,train,0,1,2,1,1,0,...,0,0,2,0,0,0,0,0,0,"[0, 2, 0, 0, 0, 0, 0, 0]"
1,今日の月も白くて明るい。昨日より雲が少なくてキレイな? と立ち止まる帰り道｡チャリなし生活も...,1,2012/08/02 23:09,train,3,0,3,0,0,0,...,1,1,0,0,2,0,0,0,0,"[1, 0, 0, 2, 0, 0, 0, 0]"


### モデル生成とベクトル化

In [8]:
# モデル生成とベクトル化 -> まあまあ時間がかかる
from sentence_transformers import SentenceTransformer, util

# モデルのロード
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
# 比較する文章
sentences = df_wrime['Sentence']
# 文章をベクトルに変換
embeddings = model.encode(sentences)

print("Vector-Transform Complete!")

Vector tranceform complete!


### 文章ベクトルと感情タグの結合

In [18]:
# 結合した結果の表示
r_emotions_array = np.array(list(df_wrime['readers_emotion_intensities']))
# list経由でなぜかうまくいく
print(f"*embeddings*       type: {type(embeddings)}, shape: {embeddings.shape}")
print(f"*r_emotions_array* type: {type(r_emotions_array)}, shape: {r_emotions_array.shape}")
vout1 = np.hstack([embeddings, r_emotions_array])
print(f"*vout1*            type: {type(vout1)}, shape: {vout1.shape}")

*embeddings*       type: <class 'numpy.ndarray'>, shape: (43200, 768)
*r_emotions_array* type: <class 'numpy.ndarray'>, shape: (43200, 8)
*vout1*            type: <class 'numpy.ndarray'>, shape: (43200, 776)


In [13]:
# 結合ベクトルの表示 -> 長いので分離
print(vout1[1])

[ 3.33150029e-02  1.20648980e-01 -1.66441165e-02  1.70662161e-02
 -7.08431602e-02 -4.44563814e-02  3.07671130e-02 -3.04284785e-02
  5.46331750e-03  8.42612162e-02  1.13711648e-01  6.16918914e-02
 -6.81294426e-02 -9.52419117e-02  1.14041857e-01 -3.30058664e-01
 -3.79631110e-02 -3.85059463e-03 -1.24080190e-02  1.70153826e-02
 -3.35456505e-02 -1.09730244e-01 -7.68772187e-03 -9.54710469e-02
  1.39225259e-01  3.90915349e-02 -2.64948327e-02  1.21489823e-01
  1.30826831e-01 -9.26807448e-02 -1.56179853e-02  6.29005730e-02
 -3.95592265e-02 -1.54046044e-02  4.77681756e-02 -5.36945388e-02
  4.76445518e-02  7.76301473e-02 -6.74038008e-02 -1.57715683e-03
  1.31837025e-01  6.96763098e-02  4.36804965e-02  2.47445107e-02
 -8.13361034e-02  7.34724626e-02 -5.06488606e-02 -6.11753725e-02
 -1.57084480e-01  6.33721575e-02 -1.33456672e-02  1.76601484e-01
 -1.09990999e-01  3.86090949e-02  5.65121360e-02  1.42349899e-01
  3.44132044e-04 -1.10288963e-01 -5.57716116e-02 -2.08153632e-02
 -9.32194442e-02 -7.80066

In [21]:
# 入出力の組を作る
vout2 = np.hstack([sentences, vout1])
print(f"*vout2*            type: {type(vout2)}, shape: {vout2.shape}")

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)

### データ出力

driveマウント準備

In [14]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.4/987.4 kB 53.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


ファイル出力

In [23]:
from datetime import datetime
import pytz
import os

# Googleドライブの保存先パスを指定
save_dir = '/content/drive/MyDrive/Colab_Outputs/'

# 保存先ディレクトリが存在しない場合は作成
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# 現在の日時を日本時間で取得
# ColabのデフォルトタイムゾーンはUTCの場合があるため、日本時間(Asia/Tokyo)を指定すると確実
tz = pytz.timezone('Asia/Tokyo')
now = datetime.now(tz)

# 日時を指定のフォーマット（例: YYYY-MM-DD_HH-MM-SS）で文字列に変換
timestamp = now.strftime("%Y-%m-%d_%H-%M-%S")

# ファイル名を生成
filename = f"output_{timestamp}.csv"
file_path = os.path.join(save_dir, filename)

# サンプルデータフレームを作成 (実際にはご自身のデータに置き換えてください)
data = vout1
df = pd.DataFrame(data)

# CSVファイルとして保存
df.to_csv(file_path, index=False)

print(f"'{file_path}' にファイルが保存されました。")


'/content/drive/MyDrive/Colab_Outputs/output_2025-11-10_16-51-48.csv' にファイルが保存されました。
